<a href="https://colab.research.google.com/github/rahiakela/advanced-natural-language-processing-with-tensorflow-2/blob/main/2-understanding-sentiment-in-natural-language/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sentiment classification with LSTMs

Sentiment classification is an oft-cited use case of NLP. Models that predict the movement of stock prices by using sentiment analysis features from tweets have shown promising results. Tweet sentiment is also used to determine customers' perceptions of brands. Another use case is processing user reviews for movies, or products on e-commerce or other websites. To see LSTMs in action, let's use a dataset of movie reviews from IMDb.

## Setup

In [15]:
# !pip install tensorflow_datasets
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing import sequence

import numpy as np

**Only if you have a GPU**

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
######## GPU CONFIGS FOR RTX 2070 ###############
## Please ignore if not training on GPU       ##
## this is important for running CuDNN on GPU ##

# check if GPU can be seen by TF
tf.config.list_physical_devices('GPU')
# enable this line if you wish to see whether GPU/CPU executes
# each command
#tf.debugging.set_log_device_placement(True)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)
###############################################

1 Physical GPUs, 1 Logical GPU


## Loading the data

When we load data using Pandas, it load the entire dataset into memory.However, sometimes data can be quite large, or spread into multiple files. In such cases, it may be too large for loading and need lots of pre-processing. Making text data ready to be used in a model requires normalization and vectorization at the very least. Often, this needs to be done outside of the TensorFlow graph using Python functions. Further, it creates issues for data pipelines in production where there is a higher chance of breakage as different dependent stages are being executed separately.

TensorFlow provides a solution for the loading, transformation, and batching
of data through the use of the tf.data package. In addition, a number of datasets are provided for download through the tensorflow_datasets package. We will use a combination of these to download the IMDb data, and perform the tokenization, encoding, and vectorization steps before training an LSTM model.

The `tfds` package comes with a number of datasets in different domains such as images, audio, video, text, summarization, and so on.

In [4]:
# see available tfds data sets
", ".join(tfds.list_builders())

'abstract_reasoning, accentdb, aeslc, aflw2k3d, ag_news_subset, ai2_arc, ai2_arc_with_ir, amazon_us_reviews, anli, arc, bair_robot_pushing_small, bccd, beans, big_patent, bigearthnet, billsum, binarized_mnist, binary_alpha_digits, blimp, bool_q, c4, caltech101, caltech_birds2010, caltech_birds2011, cars196, cassava, cats_vs_dogs, celeb_a, celeb_a_hq, cfq, chexpert, cifar10, cifar100, cifar10_1, cifar10_corrupted, citrus_leaves, cityscapes, civil_comments, clevr, clic, clinc_oos, cmaterdb, cnn_dailymail, coco, coco_captions, coil100, colorectal_histology, colorectal_histology_large, common_voice, coqa, cos_e, cosmos_qa, covid19sum, crema_d, curated_breast_imaging_ddsm, cycle_gan, deep_weeds, definite_pronoun_resolution, dementiabank, diabetic_retinopathy_detection, div2k, dmlab, downsampled_imagenet, dsprites, dtd, duke_ultrasound, emnist, eraser_multi_rc, esnli, eurosat, fashion_mnist, flic, flores, food101, forest_fires, fuss, gap, geirhos_conflict_stimuli, genomics_ood, german_credit

IMDb data is provided in three splits – training, test, and unsupervised. The training and testing splits have 25,000 rows each, with two columns. The first column is the text of the review, and the second is the label. "0" represents a review with negative sentiment while "1" represents a review with positive sentiment.

In [ ]:
# using TFDS dataset
# note: as_supervised converts dicts to tuples
imdb_train, ds_info = tfds.load(name="imdb_reviews", split="train", with_info=True, as_supervised=True)
imdb_test = tfds.load(name="imdb_reviews", split="test", as_supervised=True)

In [6]:
print(ds_info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

We can see that two keys, text and label, are available in the supervised mode. Using the as_supervised parameter is key to loading the dataset as a tuple of values. If this parameter is not specified, data is loaded and made available as dictionary keys. In cases where the data has multiple inputs, that may be preferable.

In [7]:
for example, label in imdb_train.take(1):
    print(example, '\n', label)

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) 
 tf.Tensor(0, shape=(), dtype=int64)


## Normalization and vectorization

Here, we are only going to tokenize the text into words and construct a
vocabulary, and then encode the words using this vocabulary. This is a simplified approach. There can be a number of different approaches that can be used for building additional features.

A vocabulary of the tokens occurring in the data needs to be constructed prior to vectorization. Tokenization breaks up the words in the text into individual tokens. The set of all the tokens forms the vocabulary.

Normalization of the text, such as converting to lowercase, etc., is performed along with this tokenization step. `tfds` comes with a set of feature builders for text in the `tfds.features.text` package.

First, a set of all the words in the training data needs to be created:

In [10]:
# Use the default tokenizer settings
tokenizer = tfds.deprecated.text.Tokenizer()

vocabulary_set = set()
MAX_TOKENS = 0

for example, label in imdb_train:
  some_tokens = tokenizer.tokenize(example.numpy())
  if MAX_TOKENS < len(some_tokens):
        MAX_TOKENS = len(some_tokens)
  vocabulary_set.update(some_tokens)

Using this vocabulary, an encoder can be created. TokenTextEncoder is one of three out-of-the-box encoders that are provided in tfds. Note how the list of tokens is converted into a set to ensure only unique tokens are retained in the vocabulary. The tokenizer used for generating the vocabulary is passed in, so that every successive call to encode a string can use the same tokenization scheme.

In [12]:
imdb_encoder = tfds.deprecated.text.TokenTextEncoder(vocabulary_set, tokenizer=tokenizer)
vocab_size = imdb_encoder.vocab_size

print(vocab_size, MAX_TOKENS)

93931 2525


The vocabulary has 93,931 tokens. The longest review has 2,525 tokens. That is one wordy review! Reviews are going to have different lengths.

LSTMs expect sequences of equal length. Padding and truncating operations make reviews of equal length.

In [14]:
# Lets verify tokenization and encoding works
for example, label in imdb_train.take(1):
    print(f"Encoded: \n", example)
    encoded = imdb_encoder.encode(example.numpy())
    print("Decoded: \n", imdb_encoder.decode(encoded))

Encoded: 
 tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
Decoded: 
 This was an absolutely terrible movie Don t be lured in by Christopher Walken or Michael Ironside Both are great actors but this must simply be their worst role in history Even their great acting could not redeem this movie s ridi

Tokenization and encoding were done for a small set of rows at a time. TensorFlow provides mechanisms to perform these actions in bulk over large datasets, which can be shuffled and loaded in batches. This allows very large datasets to be loaded without running out of memory during training. To enable this, a function needs to be defined that performs a transformation on a row of data. Note that multiple transformations can be chained one after the other. It is also possible to use a Python function in defining these transformations. 

For processing the review above, the following steps need to be performed:
- **Tokenization**: Reviews need to be tokenized into words.
- **Encoding**: These words need to be mapped to integers using the vocabulary.
- **Padding**: Reviews can have variable lengths, but LSTMs expect vectors of
the same length. So, a constant length is chosen. Reviews shorter than this
length are padded with a specific vocabulary index, usually 0 in TensorFlow.
Reviews longer than this length are truncated.



In [16]:
# transformation fucntions to be used with the dataset
def encode_pad_transform(sample):
    # encoding, padding and truncating
    encoded = imdb_encoder.encode(sample.numpy())
    pad = sequence.pad_sequences([encoded], padding='post', maxlen=150)

    return np.array(pad[0], dtype=np.int64)  


def encode_tf_fn(sample, label):
    encoded = tf.py_function(encode_pad_transform, inp=[sample], Tout=(tf.int64))
    encoded.set_shape([None])
    label.set_shape([])

    return encoded, label

Transforming the data is quite trivial.

In [17]:
# test the transformation on a small subset
subset = imdb_train.take(10)
tst = subset.map(encode_tf_fn)

In [18]:
for review, label in tst.take(1):
    print(review, label)
    print(imdb_encoder.decode(review))

tf.Tensor(
[10791 10817 40367 83856 44059 66762 60896 76281 86381 52497 75002 88045
 60004 91183 28061 15113 23589 45274 29445 87375 16264 77090 81275 21491
 91250 86381   596 48122 54858 75002 58898 65100   596 87375 21198 56449
  6219 63042 81275 66762 27491  9280 22022 10791 66762 70078 40367 24997
  9196 38223 84205  2447 22603  7336 29982 63572 68348 55554 35879 41569
 72364 86059 68348 49412   596  4918  4740 59976 11831 18333 65828 60033
 47683 83758 68578 11127 21426 87468 60847 91183 10817 80368 77090 28169
 29982 50099 53376 75002 28169 66762 18138 10817 77349 23289 22940 72562
 24166 42564 62669 87346 18138 82949 29445  4080 56663 81275 36517 47183
 27491 56663 60004 91183 27491 75536  4318 42564 56449 58963 34040 29022
 42565     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
This was an

Running this map over the entire dataset can be done like so:

In [19]:
#now tokenize/encode/pad all training and testing data
encoded_train = imdb_train.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
encoded_test = imdb_test.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)

While we have normalized and encoded the text of the reviews, we have not
converted it into word vectors or embeddings. This step is performed along with the model training in the next step.

## Preparing the model

In [ ]:
# Length of the vocabulary in chars
vocab_size = imdb_encoder.vocab_size # len(chars)

# The embedding dimension
embedding_dim = 64

# Number of RNN units
rnn_units = 64

#batch size
BATCH_SIZE=100

In [ ]:
def build_model_lstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model


In [ ]:
model = build_model_lstm(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (100, None, 64)           6011584   
_________________________________________________________________
lstm (LSTM)                  (100, 64)                 33024     
_________________________________________________________________
dense (Dense)                (100, 1)                  65        
Total params: 6,044,673
Trainable params: 6,044,673
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy', 'Precision', 'Recall'])

# Model Training and Evaluation

In [ ]:
# Prefetch for performance
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)

In [ ]:
model.fit(encoded_train_batched, epochs=10)

Epoch 1/10
250/250 [==============================] - 21s 84ms/step - loss: 0.4106 - accuracy: 0.8049 - precision: 0.7849 - recall: 0.8400
Epoch 2/10
250/250 [==============================] - 22s 88ms/step - loss: 0.1692 - accuracy: 0.9380 - precision: 0.9403 - recall: 0.9353
Epoch 3/10
250/250 [==============================] - 22s 88ms/step - loss: 0.1109 - accuracy: 0.9622 - precision: 0.9640 - recall: 0.9603
Epoch 4/10
250/250 [==============================] - 22s 88ms/step - loss: 0.0674 - accuracy: 0.9777 - precision: 0.9781 - recall: 0.9773
Epoch 5/10
250/250 [==============================] - 22s 87ms/step - loss: 0.0552 - accuracy: 0.9823 - precision: 0.9819 - recall: 0.9828
Epoch 6/10
250/250 [==============================] - 23s 90ms/step - loss: 0.0654 - accuracy: 0.9774 - precision: 0.9770 - recall: 0.9779
Epoch 7/10
250/250 [==============================] - 22s 89ms/step - loss: 0.0257 - accuracy: 0.9924 - precision: 0.9921 - recall: 0.9928
Epoch 8/10
250/250 [=======

In [ ]:
model.evaluate(encoded_test.batch(BATCH_SIZE))

250/250 [==============================] - 21s 82ms/step - loss: 0.9765 - accuracy: 0.8317 - precision: 0.8013 - recall: 0.8822


[0.9764885902404785,
 0.8317199945449829,
 0.8013225793838501,
 0.8821600079536438]

# BiLSTM Model

In [ ]:
# Length of the vocabulary in chars
vocab_size = imdb_encoder.vocab_size # len(chars)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 64

#batch size
BATCH_SIZE=50

In [ ]:
dropout=0.2
def build_model_bilstm(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, return_sequences=True, dropout=0.25)),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(rnn_units, dropout=0.25)),
    tf.keras.layers.Dropout(dropout),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])
  return model

In [ ]:
bilstm = build_model_bilstm(
  vocab_size = vocab_size,
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

bilstm.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (50, None, 128)           12023168  
_________________________________________________________________
dropout (Dropout)            (50, None, 128)           0         
_________________________________________________________________
bidirectional (Bidirectional (50, None, 128)           98816     
_________________________________________________________________
dropout_1 (Dropout)          (50, None, 128)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (50, 128)                 98816     
_________________________________________________________________
dropout_2 (Dropout)          (50, 128)                 0         
_________________________________________________________________
dense_1 (Dense)              (50, 1)                  

In [ ]:
bilstm.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy', 'Precision', 'Recall'])

In [ ]:
encoded_train_batched = encoded_train.batch(BATCH_SIZE).prefetch(100)

In [ ]:
bilstm.fit(encoded_train_batched, epochs=5)

Epoch 1/5
500/500 [==============================] - 68s 136ms/step - loss: 0.3763 - accuracy: 0.8266 - precision: 0.8205 - recall: 0.8362
Epoch 2/5
500/500 [==============================] - 69s 138ms/step - loss: 0.1505 - accuracy: 0.9465 - precision: 0.9463 - recall: 0.9467
Epoch 3/5
500/500 [==============================] - 69s 137ms/step - loss: 0.0701 - accuracy: 0.9761 - precision: 0.9765 - recall: 0.9757
Epoch 4/5
500/500 [==============================] - 68s 137ms/step - loss: 0.0724 - accuracy: 0.9753 - precision: 0.9747 - recall: 0.9758
Epoch 5/5
500/500 [==============================] - 69s 137ms/step - loss: 0.0416 - accuracy: 0.9859 - precision: 0.9856 - recall: 0.9862


In [ ]:
bilstm.evaluate(encoded_test.batch(BATCH_SIZE))

500/500 [==============================] - 17s 35ms/step - loss: 0.6753 - accuracy: 0.8384 - precision: 0.8459 - recall: 0.8274


[0.6753188967704773,
 0.8383600115776062,
 0.8459147810935974,
 0.8274400234222412]